<a href="https://colab.research.google.com/github/dvarelaj/nlp-miniproyecto-icesi/blob/main/analisis_sentimientos_quejas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. ANÁLISIS DE SENTIMIENTO EN QUEJAS DE SOPORTE TÉCNICO (ARL)
**Autores:** Diana Varela, Daniel García, Farid Sandoval  
**Programa:** Maestría en Inteligencia Artificial Aplicada - Universidad ICESI  

---

## CONTEXTO DEL PROYECTO
En esta actividad, se busca replicar el flujo de trabajo del notebook de reseñas de películas, pero aplicado a un dominio específico y complejo: **quejas técnicas de una ARL en español**.

A diferencia del ejemplo base, aquí enfrentamos dos retos adicionales:
* **Cambio de Idioma:** Se migra de VADER (inglés) a `pysentimiento`, un modelo basado en Deep Learning (BERT) optimizado para español.
* **Dominio Técnico:** El lenguaje corporativo y técnico presenta matices de neutralidad y negatividad que difieren del lenguaje coloquial de las películas.

In [ ]:
import pandas as pd
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from spacy.matcher import Matcher
from collections import Counter
import spacy.cli
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

Se hace etiquetado manual sobre la muestra de quejas seleccionadas para correr el modelo y poder comparar.

In [ ]:
# URL de GitHub
url_datos = 'https://raw.githubusercontent.com/dvarelaj/nlp-miniproyecto-icesi/main/quejas_anonimizadas_muestra_etiqueta.csv'
df = pd.read_csv(url_datos, delimiter=';')

# 2. PRE-PROCESAMIENTO
Para garantizar la calidad de las predicciones, realizamos un flujo de limpieza.

* **Carga de datos:** Se utiliza una muestra de **101 registros** anonimizados de la ARL.
* **Limpieza:** Se eliminan valores nulos y celdas con espacios en blanco en la descripción de la queja para evitar errores en el tokenizador.

In [ ]:
# Limpieza básica
df.dropna(subset=['descripcion_anonimizada'], inplace=True)
df = df[df['descripcion_anonimizada'].str.strip() != '']

print(f"Total de quejas para analizar: {len(df)}")

# 3. IMPLEMENTACIÓN DEL MODELO
A diferencia del modelo basado en léxicos (VADER) visto en clase, para este caso "personalizado" optamos por un **Sentiment Analyzer** basado en Transformers, ya que nuestro archivo de datos tiene quejas y peticiones en español.

Este modelo es superior para el español ya que:
1. Entiende el contexto bidireccional de las frases.
2. Detecta ironía y matices técnicos que modelos más simples ignorarían.

In [ ]:
#!pip install pysentimiento

In [ ]:
!pip install pysentimiento
import pysentimiento
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")

# Aplicamos el análisis a nuestras quejas
df['sentiment_results'] = df['descripcion_anonimizada'].apply(lambda x: analyzer.predict(x))
df['prediction'] = df['sentiment_results'].apply(lambda r: r.output)
df['compound_score'] = df['sentiment_results'].apply(lambda r: r.probas['NEG'] * -1 if r.output == 'NEG' else r.probas['POS'])

df[['descripcion_anonimizada', 'prediction']].head()

In [ ]:
# Conteo de los resultados obtenidos por el modelo
conteo_sentimientos = df['prediction'].value_counts()
print(conteo_sentimientos)

In [ ]:
# Predicciones del modelo
y_pred = [p.lower() for p in df['prediction'].tolist()]

# Obtener las etiquetas verdaderas
y_true = df['y_true'].tolist()

# Generar el reporte (requires y_true)
reporte = classification_report(y_true, y_pred)
print("### Reporte de Clasificación ###")
print(reporte)

# Calcular exactitud total (requires y_true)
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy Total: {accuracy:.2f}")

In [ ]:
# Crear la matriz de confusión
labels = ['neg', 'neu', 'pos']
cm = confusion_matrix(y_true, y_pred, labels=labels)

# Graficar
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicción del Modelo (pysentimiento)')
plt.ylabel('Etiqueta Real (Manual)')
plt.title('Matriz de Confusión: Quejas ARL')
plt.show()

# 4. ANÁLISIS DE RESULTADOS

Una vez ejecutado el modelo sobre las 101 quejas, se presentan los siguientes hallazgos críticos basados en las métricas obtenidas:

### A. RENDIMIENTO DEL MODELO
El modelo supera significativamente la línea base del 50%, logrando un **Accuracy del 71%**. Sin embargo, el rendimiento es asimétrico:
* **Alta Precisión en Negativos (0.86):** El modelo es muy efectivo detectando la molestia real del usuario en temas de fallas técnicas.
* **Debilidad en Neutrales (Precision 0.37):** Existe una confusión importante entre quejas informativas y consultas de trámites.



### B. OBSERVACIONES LINGÜÍSTICAS
Como hallazgo, identificamos un fenómeno de **"Ambigüedad de Dominio"**:
* Términos como *"mantenimiento"*, *"portal no carga"* o *"no permite ingresar"* son clasificados frecuentemente como **NEG** (negativos) por el modelo.
* Lingüísticamente, estas frases podrían ser informativas (Neutrales), pero en el ecosistema de una ARL, el modelo interpreta correctamente la carga emocional de frustración implícita en la imposibilidad de realizar una tarea.

### C. COMPARACIÓN CON EL MODELO BASE
* **Desbalance Natural:** A diferencia del dataset de películas (balanceado 50/50), las quejas de la ARL están **naturalmente desbalanceadas hacia lo negativo**.
* **Ausencia de Positivos:** La ausencia total de sentimientos positivos (**POS = 0**) refleja la realidad de los canales de soporte: el usuario rara vez se comunica para expresar satisfacción en un canal de reclamos.

In [ ]:
# Gráfico de barras para visualizar el desbalance de sentimientos
plt.figure(figsize=(7,5))
sns.countplot(x='prediction', data=df, palette='viridis')
plt.title('Distribución de Predicciones: Dominio ARL')
plt.show()